In [2]:
%load_ext sql

In [4]:
%sql sqlite:///./publications.db

'Connected: @./publications.db'

## Challenge 1 - Most Profiting Authors

In this challenge let's have a close look at the bonus challenge of the previous *SQL SELECT* lab -- **who are the top 3 most profiting authors**? Even if you have solved or think you have solved that problem in the previous lab, please still complete this challenge because the step-by-step guidances are helpful to train your problem-solving thinking.

In order to solve this problem, it is important for you to keep the following points in mind:

* In table `sales`, a title can appear several times. The royalties need to be calculated for each sale.

* Despite a title can have multiple `sales` records, the advance must be calculated only once for each title.

* In your eventual solution, you need to sum up the following profits for each individual author:
    * All advances which is calculated exactly once for each title.
    * All royalties in each sale.

Therefore, you will not be able to achieve the goal with a single SELECT query. Instead, you will need to follow several steps in order to achieve the eventual solution. Below is an overview of the steps:

1. Calculate the royalty of each sale for each author.

1. Using the output from Step 1 as a sub-table, aggregate the total royalties for each title for each author.

1. Using the output from Step 2 as a sub-table, calculate the total profits of each author by aggregating the advances and total royalties of each title.

Below we'll guide you through each step. In your `solutions.ipynb`, please include the SELECT queries of each step so that your TA can review your problem-solving process.

### Step 1: Calculate the royalties of each sales for each author

Write a SELECT query to obtain the following output:

* Title ID
* Author ID
* Royalty of each sale for each author
    * The formula is:
        ```
        sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100
        ```
    * Note that `titles.royalty` and `titleauthor.royaltyper` are divided by 100 respectively because they are percentage numbers instead of floats.

In the output of this step, each title may appear more than once for each author. This is because a title can have more than one sales.

In [19]:
%%sql
select ti.title_id as title_id,
       ta.au_id as author_id, 
       coalesce(ti.price * sa.qty * ti.royalty / 100 * ta.royaltyper / 100, 0) as sales_royalty
from titles ti
left join titleauthor ta on ta.title_id = ti.title_id
left join sales sa on ti.title_id = sa.title_id


 * sqlite:///./publications.db
Done.


title_id,author_id,sales_royalty
BU1032,213-46-8915,3.9979999999999993
BU1032,213-46-8915,7.995999999999999
BU1032,409-56-7008,5.996999999999999
BU1032,409-56-7008,11.993999999999998
BU1111,267-41-2394,11.95
BU1111,724-80-9391,17.925
BU2075,213-46-8915,25.116000000000003
BU7832,274-80-9391,29.984999999999996
MC2222,712-45-1867,23.987999999999996
MC3021,722-51-5454,13.455


In [18]:
%%sql
-- # Exploramos el título que no tiene un autor asignado en la tabla titleauthor
select * from titles ti

left join titleauthor ta on ta.title_id = ti.title_id
where ti.title_id = 'MC3026'


 * sqlite:///./publications.db
Done.


title_id,title,type,pub_id,price,advance,royalty,ytd_sales,notes,pubdate,au_id,title_id_1,au_ord,royaltyper
MC3026,The Psychology of Computer Cooking,UNDECIDED,0877,None,None,None,None,None,2014-11-07 10:39:37,None,None,None,None


### Step 2: Aggregate the total royalties for each title for each author

Using the output from Step 1, write a query to obtain the following output:

* Title ID
* Author ID
* Aggregated royalties of each title for each author
    * Hint: use the *SUM* subquery and group by both `au_id` and `title_id`

In the output of this step, each title should appear only once for each author.

In [55]:
%%sql
select ti.title_id as title_id,
       ta.au_id as author_id,
       sum(ti.price * sa.qty * ti.royalty / 100 * ta.royaltyper / 100) as sales_royalty,
       ti.advance 
from titleauthor ta
left join titles ti on ta.title_id = ti.title_id
left join sales sa on ti.title_id = sa.title_id
group by author_id, ti.title_id

 * sqlite:///./publications.db
Done.


title_id,author_id,sales_royalty,advance
PS3333,172-32-1176,29.984999999999996,2000
BU1032,213-46-8915,11.993999999999998,5000
BU2075,213-46-8915,25.116000000000003,10125
PC1035,238-95-7766,110.16,7000
BU1111,267-41-2394,11.95,5000
TC7777,267-41-2394,8.994,8000
BU7832,274-80-9391,29.984999999999996,5000
BU1032,409-56-7008,17.990999999999996,5000
PC8888,427-17-2319,50,8000
TC7777,472-27-2349,8.994,8000


### Step 3: Calculate the total profits of each author

Now that each title has exactly one row for each author where the advance and royalties are available, we are ready to obtain the eventual output. Using the output from Step 2, write a query to obtain the following output:

* Author ID
* Profits of each author by aggregating the advance and total royalties of each title

Sort the output based on a total profits from high to low, and limit the number of rows to 3.


In [71]:
%%sql
with table2 as (select ti.title_id as title_id,
       ta.au_id as author_id,
       sum(ti.price * sa.qty * ti.royalty / 100 * ta.royaltyper / 100) as sales_royalty,
       ti.advance 
from titleauthor ta
left join titles ti on ta.title_id = ti.title_id
left join sales sa on ti.title_id = sa.title_id
group by author_id, ti.title_id)

select author_id,
       coalesce(sales_royalty + advance, 0) as total_profit_by_author

from table2
group by author_id
order by total_profit_by_author desc
limit 3



 * sqlite:///./publications.db
Done.


author_id,total_profit_by_author
722-51-5454,15021.528
899-46-2035,15007.176
846-92-7186,8050


## Challenge 2

Elevating from your solution in Challenge 1 , create a table named `most_profiting_authors` to hold the data about the most profiting authors. The table should have 2 columns:

* `au_id` - Author ID
* `profits` - The profits of the author aggregating the advances and royalties

Include your solution in `solutions.ipynb`.

In [92]:
%%sql
create table most_profiting_authors as
with table2 as (select ti.title_id as title_id,
       ta.au_id as author_id,
       sum(ti.price * sa.qty * ti.royalty / 100 * ta.royaltyper / 100) as sales_royalty,
       ti.advance 
from titleauthor ta
left join titles ti on ta.title_id = ti.title_id
left join sales sa on ti.title_id = sa.title_id
group by author_id, ti.title_id)

select author_id,
       round(coalesce(sales_royalty + advance, 0)) as total_profit_by_author

from table2
group by author_id


 * sqlite:///./publications.db
Done.


[]

In [93]:
%%sql
select * from most_profiting_authors limit 3

 * sqlite:///./publications.db
Done.


author_id,total_profit_by_author
172-32-1176,2030.0
213-46-8915,5012.0
238-95-7766,7110.0
